In [8]:
import pandas as pd
import numpy as np
import datetime
import time
import itertools
from functools import wraps

import plotly.graph_objects as go
import plotly.express as px
import dash
from dash import dcc
from dash import html
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

import ibmdata

import ruptures as rpt

from concurrent.futures import ThreadPoolExecutor, as_completed

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

Dataframes must:
- have a column named "date"
- must be ordered by date column
- must have a column of interest ie. yield, mean, etc. There can't be too many variations for this. Might have to figure out a way to rename the column of interest into a single constant name.


## Timer Wrapper

In [3]:
def timeit(method):
    @wraps(method)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = method(*args, **kwargs)
        end_time = time.time()
        print(f"{method.__name__} => {(end_time-start_time)} s")

        return result

    return wrapper

# Object

In [27]:
class cpd:
    def __init__(self):
        self.outputdf = pd.DataFrame(columns=['parameter','lot_id','date'])
    
    def pull_ilt_data(self, PARM, DAYSBACK = 365, fc = ('Q5','Q6','XQ','X2')):
        fc = str(fc)
        QUERY = f"""
        SELECT LEFT(lot_Id,5) AS lot_id, family_Code, MIN(Last_test_date) AS date, 
        tp.parm_Label as parameter, AVG(weighted_Mean) as mean 
        FROM DMIW.PTileWaferFact ptwf
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testparmkey = ptwf.testparmkey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = ptwf.testedWaferKey
        WHERE Last_test_date >= (current date - {DAYSBACK} days)
        AND Tech_id = '7HPP' 
        AND weighted_Mean is not null AND abs(weighted_Mean) < 1e25 
        AND ( tp.parm_Label LIKE '{PARM}' )
        AND family_Code IN '{fc}'
        GROUP BY LEFT(lot_Id,5), family_Code, tp.parm_Label
        ORDER BY MIN(Last_test_date), LEFT(lot_Id,5)
        """
        df = ibmdata.isdw.query(QUERY)
        df['mean'] = df['mean'].astype(float).round(2)
        return df
    
    def change_point_plot(self, file_name, df, points, result, parm_name, cp_date, cp_lot):
        COLOR_CYCLE = ["#4286f4", "#f44174"]
        colorBuffer = itertools.cycle(COLOR_CYCLE)
        
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.set_theme(style="dark")
        sns.despine()
        x_labels = df[['lot_id','date']].astype('str').agg(' '.join, axis=1)
        
        num_points = len(points)
        plt.plot(range(num_points), points)
        plt.scatter(range(num_points), points)
        
        start = 0
        for stop, color in zip(result, colorBuffer):
            left = max(0, start - 0.5)
            right = stop - 0.5
            plt.hlines(y=np.median(points[start:stop]), 
                        xmin=left, 
                        xmax=right,
                        color='r',
                        linestyle='--')
            plt.axvspan(left, right, facecolor=color, alpha=0.2, zorder=0)
            start = stop
        
        plt.title(f'Most recent change point for {parm_name} occured on {cp_date} with lot {cp_lot}')
        plt.ylabel(parm_name)

        lower_bound = np.percentile(points, 25)
        upper_bound = np.percentile(points, 75)
        ymin = lower_bound - 1.5*(upper_bound-lower_bound)
        ymax = upper_bound + 1.5*(upper_bound-lower_bound)
        plt.ylim(ymin, ymax)
        plt.xlim(0, num_points-1)
        plt.xticks(x_labels.index.values[result[:-1]], x_labels[result[:-1]], rotation=90, fontsize=12)

        plt.savefig(file_name, format='pdf',bbox_inches = "tight", pad_inches=0.5)
        plt.close('all')
        return
    
    def change_point_detector(self,pp, df, rbf_pen = 10, TIMEDELAY = 30, var='mean',date='date'):
        #Empty DataFrame
        if df.empty:
            print('The df is empty')
            return 
        if len(df) < 5:
            print('Not enough data points')
            return 

        # Find change point index
        points = np.array(df[var])
        model="rbf"
        algo = rpt.Pelt(model=model).fit(points)
        result = algo.predict(pen=rbf_pen)
        
        if len(result) < 2:
            print('No Change Points At All')
            return 
        else:
            cp = result[-2]

        #find change point date
        parm_name = df.parameter[0]
        cp_lot = df.iloc[cp]['lot_id']
        cp_date = df.iloc[cp]['date']
        today = datetime.date.today()
        delta = datetime.timedelta(days = TIMEDELAY)
        if today-cp_date < delta:
            
            # Save result to csv
            self.outputdf = self.outputdf.append(pd.DataFrame([[parm_name, cp_lot, cp_date]], 
                                                              columns=['parameter','lot_id','date']))
            print("Recent Change Point Detected")
            return {'df': df, 'points': points, 'result': result, 
                    'parm_name': parm_name, 'cp_date':cp_date, 'cp_lot': cp_lot}
        else:
            print("No Recent Change Points")
            return 

    def param_cpd(self, param, pp, rbf_pen=5, TIMEDELAY=30, DAYSBACK=365, fc=('Q5','Q6','XQ','X2')):
        
        # Pull data specific to param
        df = self.pull_ilt_data(param, DAYSBACK, fc)
        
        # Run CPD algorithm
        return self.change_point_detector(pp, df, TIMEDELAY=TIMEDELAY, rbf_pen=rbf_pen)
       
    @timeit
    def ilt_cpd_async(self, pp, parmlist, rbf_pen=5, TIMEDELAY=30, DAYSBACK=365, fc=('Q5','Q6','XQ','X2')):
        
        # Run analysis asynchronously
        with ThreadPoolExecutor(max_workers=17) as executor:
            return executor.map(self.param_cpd, 
                                parmlist, 
                                itertools.repeat(pp),
                                itertools.repeat(rbf_pen), 
                                itertools.repeat(TIMEDELAY), 
                                itertools.repeat(DAYSBACK), 
                                itertools.repeat(fc), 
                                timeout=60)

# Main Script

In [5]:
%load_ext memory_profiler
from memory_profiler import profile

In [6]:
levels_dict = {"M1": "M1", "M5": "M5", "MF": "LT"}

DAYSBACK=365
TIMEDELAY=200

def run_cpd(level, family_code):
    params = pd.read_csv(f'./Parms/{level}parms.csv', header=None, names=["Parameter", "Owner"])
    params["Parameter"] = params["Parameter"].astype('str') + f"_{levels_dict[level]}"

    # open pdf to save plots
    pdfFile = PdfPages(f'./CPDResults/{level}CPDGraphs.pdf')

    # initialize cpd object
    model = cpd()

    # run analysis on all parameters
    model.ilt_cpd(pp=pdfFile, parmlist=params["Parameter"][:5], rbf_pen=5, 
                  TIMEDELAY=TIMEDELAY, DAYSBACK=DAYSBACK, fc=(family_code))

    # save results to csv
    model.outputdf.merge(
        params, 
        left_on='parameter', 
        right_on='Parameter'
    ).drop(
        columns=['Parameter']
    ).rename(
        columns={1:'category'}
    ).to_csv(
        f"./CPDResults/{level.lower()}cpd_{family_code}.csv", 
        index=False
    )

In [13]:
run_cpd("M1", "Q6")

The df is empty
None
The df is empty
None
The df is empty
None
The df is empty
None
The df is empty
None
36.00478482246399


In [15]:
run_cpd("M5", 'Q6')

Resent Change Point Detected
{'df':    lot_id family_code        date              parameter    mean
0   A1FRF          Q6  2021-07-19  CMBSRP_M3_DUT1_RES_M5  125.41
1   A1F18          Q6  2021-07-20  CMBSRP_M3_DUT1_RES_M5  117.25
2   A1DXC          Q6  2021-07-26  CMBSRP_M3_DUT1_RES_M5  132.02
3   A1DZA          Q6  2021-07-26  CMBSRP_M3_DUT1_RES_M5  129.37
4   A1F38          Q6  2021-07-26  CMBSRP_M3_DUT1_RES_M5  122.42
..    ...         ...         ...                    ...     ...
84  A273N          Q6  2022-07-02  CMBSRP_M3_DUT1_RES_M5  114.39
85  A2C6U          Q6  2022-07-13  CMBSRP_M3_DUT1_RES_M5  108.93
86  A2CG2          Q6  2022-07-13  CMBSRP_M3_DUT1_RES_M5  111.36
87  A2ABD          Q6  2022-07-14  CMBSRP_M3_DUT1_RES_M5  114.72
88  A2CG1          Q6  2022-07-14  CMBSRP_M3_DUT1_RES_M5  111.38

[89 rows x 5 columns], 'points': array([125.41, 117.25, 132.02, 129.37, 122.42, 119.57, 118.86, 125.35,
       120.24, 129.39, 130.54, 137.82, 121.89, 131.87, 115.89, 110.34,
       1

In [32]:
run_cpd("MF", 'Q6')

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Recent Change Points
No Recent Change Points
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Poin

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points

...

/var/folders/r0/zl9449s534s95kg41vhmlkwm0000gn/T/ipykernel_22110/700704262.py:84: UserWarning: Attempting to set identical bottom == top == -10.0 results in singular transformations; automatically expanding.
  plt.ylim(ymin, ymax)


No Recent Change Points
No Recent Change Points
No Recent Change Points
No Recent Change Points
No Recent Change Points
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Recent Change Points
No Recent Change Points
No Recent Change Points
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Recent Change Points
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Poin

No Change Points At All
No Change Points At All
No Recent Change Points
No Recent Change Points
No Recent Change Points
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Recent Change Points
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Recent Change Points
No Change Points At All
No Change Points At All
No Recent Change Points
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Poin

The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df i

The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df i

In [35]:
run_cpd("M1", "X2")

The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
The df is empty
The df is empty
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Chang

t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
t
No Change Points At All
No Change Points At All
t
t
t
t
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
t
t
t
t
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
t
t
t
t
t
t
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At 

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
t
t
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Chang

No Change Points At All
No Change Points At All
t
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
t
t
No Change Points At All
No Change Points At All
t
t
No Change Points At All
t
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
t
No Change Points At All
t
No Change Points At All
No Change Points At All
t
t
No Change Points At All
No Change Points At All
No Change Points At All
t
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
t
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No C

The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
No Change Points At All
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The 

In [36]:
run_cpd("M5", "X2")

t
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
t
No Change Points At All
No Change Points At All
t
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
t
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
t
t
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No

/var/folders/r0/zl9449s534s95kg41vhmlkwm0000gn/T/ipykernel_22110/700704262.py:84: UserWarning: Attempting to set identical bottom == top == -10.0 results in singular transformations; automatically expanding.
  plt.ylim(ymin, ymax)


No Change Points At All
t
t
t
No Change Points At All
No Change Points At All
t
t
No Change Points At All
t
t
t
No Change Points At All
No Change Points At All
t
t
No Change Points At All
t
t
t
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
t
No Change Points At All
t
t
t
No Change Points At All
t
t
t
t
t
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
t
t
No Change Points At All
t
t
t
No Change Points At All
t
t
No Change Points A

The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All


The df is empty
No Change Points At All
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty


...

No Change Points At All
t
t
t
No Change Points At All
t
t
t
t


/var/folders/r0/zl9449s534s95kg41vhmlkwm0000gn/T/ipykernel_22110/700704262.py:84: UserWarning: Attempting to set identical bottom == top == -10.0 results in singular transformations; automatically expanding.
  plt.ylim(ymin, ymax)


No Change Points At All
t
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
t
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
The df is empty
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
The df is empty
The df is empty
No Change Points At All
The df is empty
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
Th

No Change Points At All
No Change Points At All
No Change Points At All
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty


The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
No Change Points At All
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
No Change Points At All
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
Th

In [38]:
run_cpd("MF", "X2")

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All


No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points

...

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
The df is empty
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All


t
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Cha

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
The df is empty
No Change Points A

The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty


No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
The df is empty
The df is empty
No Change Points At All
No Change Points At All
No Chang

In [11]:
m5parms = pd.read_csv('./Parms/M5parms.csv', header=None, names=["Parameter", "Owner"])
m5parms["Parameter"] = m5parms["Parameter"].astype('str') + "_M5"
m5parms

,Parameter,Owner
0,CMBSRP_M3_DUT1_RES_M5,BEOL
1,CMBSRP_M3_DUT2_RES_M5,BEOL
2,CMBSRP_M3_DUT1_LKG_M5,BEOL
3,CMBSRP_M3_DUT2_LKG_M5,BEOL
4,CMBSRP_M4_DUT1_RES_M5,BEOL
...,...,...
2524,ROR9T4FMDBFO1~Idda_075_M5,Device
2525,ROR9T4FMDBFO1~Iddq_075_M5,Device
2526,ROSL9T4FMDBFO1~Freq_075_M5,Device
2527,ROSL9T4FMDBFO1~Idda_075_M5,Device


In [26]:
mfparms = pd.read_csv('./Parms/MFparms.csv', header=None, names=["Parameter", "Owner"])
mfparms[0] = mfparms[0].astype('str') + "_LT"
mfparms

,0,1
0,PCHK_0.1V_01_LT,BEOL
1,PCHK_0.1V_02_LT,BEOL
2,PCHK_0.1V_03_LT,BEOL
3,PCHK_0.1V_04_LT,BEOL
4,PCHK_0.1V_05_LT,BEOL
...,...,...
4474,P10_VMA_HBiso_p64MA~Short_LT,BEOL
4475,P10_VMA_HBiso_p72MA~Short_LT,BEOL
4476,TDDB_BL_120K_HB_m~Vbd_LT_LT,REL
4477,TDDB_BL_120K_HB_m~Vbd_i_LT_LT,REL


In [34]:
m1parms

NameError: name 'm1parms' is not defined

In [47]:
%memit run_cpd("M1", "X2")

The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
The df is empty
The df is empty
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Chang

t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
t
No Change Points At All
No Change Points At All
t
t
t
t
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
t
t
t
t
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
No Change Points At All
t
t
t
t
t
t
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At 

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
t
t
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Chang

No Change Points At All
No Change Points At All
t
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
t
t
No Change Points At All
No Change Points At All
t
t
No Change Points At All
t
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
t
No Change Points At All
t
No Change Points At All
No Change Points At All
t
t
No Change Points At All
No Change Points At All
No Change Points At All
t
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
t
t
No Change Points At All
No Change Points At All
t
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No C

The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
No Change Points At All
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The df is empty
The df is empty
t
The df is empty
The df is empty
The 

In [48]:
c = cpd()
dfs = []
starttime=time.time()
for p in m1parms[0]:
    df = c.pull_ilt_data(p, DAYSBACK, "X2")
    dfs.append(df)
    print(time.time()-starttime)

4.266013145446777
8.49347710609436
12.793224096298218
16.576448917388916
20.576963901519775
24.364443063735962
28.355399131774902
32.605859994888306
37.78220891952515
42.40182399749756
46.68899703025818
51.71150088310242
56.51548409461975
60.38094997406006
64.33986401557922
68.30035591125488
72.89968514442444
78.12685704231262
82.65374207496643
86.32774114608765
90.62209606170654
94.50613117218018
98.13188695907593
102.292151927948
106.38831496238708
110.29650092124939
114.67910385131836
119.08747601509094
122.8466739654541
126.79659700393677
131.69117498397827
136.9050920009613
142.33151197433472
147.96503686904907
154.21081590652466
160.18033814430237
165.3093740940094
170.61346006393433
175.82939505577087
181.24746894836426
185.6022219657898
190.16065096855164
195.48031997680664
201.31537699699402
206.86121582984924
212.77096581459045
219.1394658088684
225.18793892860413
230.83521914482117
235.92611289024353
241.39882683753967
246.07493591308594
250.91864013671875
255.9576952457428


2761.0031111240387
2766.1229169368744
2771.1083920001984
2775.907593011856
2781.450656890869
2786.6074249744415
2792.6447031497955
2799.1139760017395
2804.422035217285
2810.1466541290283
2816.136778831482
2820.909003019333
2825.5099098682404
2830.943382024765
2836.016387939453
2841.2752878665924
2847.327529191971
2852.9579548835754
2858.5616538524628
2863.4981138706207
2869.5975620746613
2875.1824040412903
2880.396115064621
2885.725666999817
2891.9634919166565
2897.816386938095
2903.021022081375
2907.8625960350037
2913.05215716362
2918.203136920929
2923.0946259498596
2928.444447994232
2933.4997420310974
2938.678900241852
2943.6704847812653
2948.5001561641693
2953.7812571525574
2958.6342828273773
2963.9496731758118
2969.1246750354767
2973.8922860622406
2978.9035999774933
2983.4414100646973
2988.4595260620117
2993.965444087982
3000.216595172882
3006.2752759456635
3012.123564004898
3018.252918958664
3023.6384251117706
3028.578407049179
3033.9096369743347
3038.6992568969727
3043.8171141147

5264.462666988373
5269.697299003601
5274.331487894058
5280.062403917313
5286.320535898209
5291.743968009949
5297.166203022003
5302.91773891449
5307.949727058411
5313.046950101852
5319.1690900325775
5324.941421031952
5331.438593864441
5337.465633869171
5343.322045087814
5347.5582818984985
5351.996819019318
5358.629987001419
5365.170850038528
5370.483000993729
5375.939020872116
5381.041712999344
5385.94970202446
5391.289176940918
5396.002390861511
5400.915749073029
5407.855407953262
5413.275439977646
5418.9012360572815
5423.230375051498
5427.9144151210785
5433.759435892105
5440.723433017731
5445.955718040466
5451.135422229767
5456.699994087219
5462.146084070206
5467.262628078461
5473.782424926758
5480.494480133057
5486.616574048996
5493.892536878586
5501.89984703064
5509.003684997559
5515.078524112701
5521.219661951065
5526.863047122955
5532.394463062286
5538.103038072586
5543.737688064575
5548.656007051468
5553.6856989860535
5557.984699964523
5562.569548845291
5567.496109962463
5572.583

7853.6243669986725
7858.857157945633
7863.5657370090485
7868.187292098999
7873.108705043793
7878.123925924301
7883.262628078461
7888.127948045731
7892.885926961899
7898.07821893692
7904.834234952927
7911.376999139786
7917.5259890556335
7923.677566051483
7929.376587867737
7937.009659051895
7942.323871135712
7948.073179960251
7953.541447162628
7959.4382219314575
7964.050666093826
7969.381770849228
7974.291283130646
7980.230911970139
7985.116954088211
7990.6862099170685
7997.002756118774
8002.541115999222
8007.2769429683685
8011.84285402298
8017.216084957123
8023.945811033249
8029.17059803009
8034.130393028259
8039.069475889206
8044.613258838654
8050.015341997147
8054.797963857651
8059.791574954987
8064.636004924774
8069.617764949799
8074.825943946838
8079.650834083557
8084.765424013138
8089.895747900009
8095.245216846466
8100.402780056
8105.766974210739
8110.991780042648
8115.399658203125
8121.62562084198
8127.369575023651
8134.183588027954
8140.2599630355835
8147.313434123993
8155.30656

10878.446362018585
10883.334595918655
10888.273866891861
10893.956140041351
10901.229645967484
10908.510262966156
10914.856168031693
10921.20516705513
10927.113102912903
10933.194453001022
10938.411348819733
10943.519500017166
10949.175419092178
10954.963081121445
10960.131116867065
10965.237862110138
10970.044475078583
10975.78140091896
10980.790210008621
10985.299638986588
10989.493654966354
10993.71877503395
10998.314181089401
11002.627889156342
11007.931378126144
11013.571064949036
11018.47579407692
11024.015535116196
11029.408159971237
11033.838829994202
11037.73850107193
11042.019680023193
11047.208700180054
11052.88799405098
11059.152865886688
11064.874140024185
11070.819368124008
11076.577599048615
11081.473211050034
11086.387620925903
11091.283123016357
11095.709160089493
11100.805156946182
11105.23516702652
11109.100650072098
11114.235866069794
11120.887698173523
11126.95131111145
11133.507152080536
11138.69523191452
11143.82169508934
11149.19333910942
11153.892478227615
1115

13533.76360797882
13539.566411972046
13544.881530046463
13550.612243175507
13556.265372991562
13562.507989883423
13568.43867611885
13575.908993959427
13582.193114995956
13588.095636844635
13593.826704025269
13601.594697237015
13608.880597829819
13614.823049783707
13621.785395145416
13627.528194904327
13635.230664014816
13641.560286045074
13649.581681966782
13656.907303094864
13663.524766206741
13669.705705165863
13675.030853033066
13681.017102956772
13687.413266181946
13693.053707838058
13700.75562095642
13709.782582998276
13718.483258008957
13726.685948133469
13736.162588119507
13744.635843992233
13756.33574795723
13766.174153804779
13775.876590013504
13784.724941015244
13793.245469093323
13801.394562959671
13809.899637937546
13816.81718325615
13824.03813791275
13834.149552106857
13841.298258066177
13849.044141054153
13856.097243070602
13862.805986881256
13870.592508077621
13876.894934892654
13883.737313985825
13889.86788225174
13895.703533887863
13903.39118719101
13909.860301971436
1

17229.28333210945
17235.094798088074
17241.68682694435
17247.214988946915
17252.654236078262
17257.879456996918
17262.773311138153
17267.79269504547
17273.294178962708
17278.84543800354
17283.69793200493
17288.442838907242
17293.56755590439
17299.120655059814
17304.44250512123
17309.17124390602
17314.092514038086
17318.521952867508
17323.19504594803
17328.409659147263
17333.546079158783
17338.271898031235
17343.96937894821
17350.36946797371
17356.570407152176
17361.89264702797
17368.29390501976
17374.765528917313
17381.086971998215
17387.651386022568
17392.46933913231
17398.54170513153
17405.631893873215
17412.86322593689
17419.386733055115
17426.728780031204
17433.671692848206
17440.480185985565
17446.591390132904
17452.31225991249
17457.63302898407
17462.864530086517
17468.200464963913
17473.81975221634
17479.541342020035
17485.29835510254
17491.532433986664
17497.88177394867
17505.190721035004
17511.043327093124
17516.53130698204
17522.458975076675
17528.392770051956
17534.265667915

In [49]:
c = cpd()
starttime=time.time()
pp = PdfPages('./CPDResults/M1CPDGraphs.pdf')
for df in dfs:
    c.change_point_detector(pp,df, TIMEDELAY = 200, rbf_pen = 10)
    print(time.time()-starttime)

The df is empty
0.0028870105743408203
The df is empty
0.002910137176513672
The df is empty
0.00292205810546875
The df is empty
0.0029342174530029297
The df is empty
0.002946138381958008
The df is empty
0.0029561519622802734
The df is empty
0.0029671192169189453
The df is empty
0.002978086471557617
The df is empty
0.002988100051879883
No Change Points At All
0.003557920455932617
No Change Points At All
0.003931999206542969
No Change Points At All
0.004523038864135742
No Change Points At All
0.004857301712036133
No Change Points At All
0.005170106887817383
No Change Points At All
0.005479097366333008
The df is empty
0.005498170852661133
The df is empty
0.005510091781616211
No Change Points At All
0.0058100223541259766
No Change Points At All
0.00613713264465332
No Change Points At All
0.006432056427001953
No Change Points At All
0.006727933883666992
No Change Points At All
0.007018089294433594
No Change Points At All
0.007310152053833008
No Change Points At All
0.007595062255859375
No Ch

3.252685070037842
No Change Points At All
3.2561752796173096
No Change Points At All
3.2593071460723877
No Change Points At All
3.2619481086730957
t
3.3618619441986084
No Change Points At All
3.3644139766693115
No Change Points At All
3.3668901920318604
No Change Points At All
3.369415044784546
t
3.4807701110839844
t
3.5923242568969727
t
3.704495906829834
No Change Points At All
3.70774507522583
No Change Points At All
3.7105941772460938
No Change Points At All
3.7131190299987793
No Change Points At All
3.7160420417785645
No Change Points At All
3.7186172008514404
No Change Points At All
3.7212040424346924
No Change Points At All
3.7237441539764404
t
3.825515031814575
t
3.9260201454162598
No Change Points At All
3.9294211864471436
t
4.028353214263916
t
4.134536027908325
No Change Points At All
4.139033079147339
No Change Points At All
4.142582178115845
No Change Points At All
4.145183086395264
t
4.24745512008667
No Change Points At All
4.250031232833862
No Change Points At All
4.252528

6.914260149002075
No Change Points At All
6.917530059814453
No Change Points At All
6.922107934951782
No Change Points At All
6.924899101257324
No Change Points At All
6.927560091018677
No Change Points At All
6.930210113525391
No Change Points At All
6.9327099323272705
No Change Points At All
6.935311317443848
No Change Points At All
6.937865972518921
No Change Points At All
6.940356254577637
No Change Points At All
6.942895174026489
No Change Points At All
6.945433139801025
No Change Points At All
6.948010206222534
No Change Points At All
6.950520277023315
t
7.053062915802002
t
7.165124177932739
t
7.281394004821777
t
7.397945165634155
No Change Points At All
7.401149034500122
No Change Points At All
7.4037721157073975
No Change Points At All
7.407663106918335
No Change Points At All
7.410501956939697
No Change Points At All
7.413219213485718
No Change Points At All
7.4159581661224365
No Change Points At All
7.4187541007995605
No Change Points At All
7.421444892883301
No Change Points

11.959468126296997
No Change Points At All
11.96281099319458
No Change Points At All
11.965338945388794
t
12.07062315940857
No Change Points At All
12.073189973831177
No Change Points At All
12.075762033462524
t
12.189359188079834
No Change Points At All
12.194042205810547
No Change Points At All
12.196531057357788
No Change Points At All
12.199446201324463
No Change Points At All
12.202373027801514
No Change Points At All
12.205239057540894
No Change Points At All
12.208010196685791
No Change Points At All
12.210772037506104
No Change Points At All
12.213479042053223
No Change Points At All
12.216243028640747
No Change Points At All
12.219030141830444
No Change Points At All
12.221790075302124
No Change Points At All
12.224487066268921
No Change Points At All
12.227265119552612
No Change Points At All
12.229959964752197
No Change Points At All
12.232738256454468
t
12.340154886245728
No Change Points At All
12.34280514717102
No Change Points At All
12.345516204833984
No Change Points A

No Change Points At All
13.065763235092163
No Change Points At All
13.069230079650879
No Change Points At All
13.071648120880127
No Change Points At All
13.074778079986572
No Change Points At All
13.077907085418701
No Change Points At All
13.080428123474121
No Change Points At All
13.082695960998535
No Change Points At All
13.085092306137085
No Change Points At All
13.087368965148926
No Change Points At All
13.089884042739868
No Change Points At All
13.092881917953491
No Change Points At All
13.095808029174805
No Change Points At All
13.099633932113647
No Change Points At All
13.102725982666016
No Change Points At All
13.105109214782715
No Change Points At All
13.107762098312378
No Change Points At All
13.110183000564575
No Change Points At All
13.112953186035156
No Change Points At All
13.115401029586792
No Change Points At All
13.117815256118774
No Change Points At All
13.120375156402588
No Change Points At All
13.12288212776184
No Change Points At All
13.12524700164795
The df is emp

...

No Change Points At All
14.361656188964844
No Change Points At All
14.368845224380493
No Change Points At All
14.374425172805786
No Change Points At All
14.37969708442688
No Change Points At All
14.384478092193604
No Change Points At All
14.389353036880493
No Change Points At All
14.39393424987793
No Change Points At All
14.398546934127808
No Change Points At All
14.402605056762695
No Change Points At All
14.406432151794434
No Change Points At All
14.409658193588257
No Change Points At All
14.41313099861145
No Change Points At All
14.416284084320068
No Change Points At All
14.419467210769653
No Change Points At All
14.422580003738403
No Change Points At All
14.425678253173828
No Change Points At All
14.428705215454102
No Change Points At All
14.431791305541992
No Change Points At All
14.43484115600586
No Change Points At All
14.437947988510132
No Change Points At All
14.440979957580566
No Change Points At All
14.444086074829102
No Change Points At All
14.44710898399353
No Change Points

15.048208951950073
t
15.154808282852173
t
15.262640953063965
No Change Points At All
15.266391038894653
No Change Points At All
15.270148277282715
No Change Points At All
15.272901058197021
No Change Points At All
15.275773048400879
No Change Points At All
15.278817176818848
No Change Points At All
15.281288146972656
No Change Points At All
15.28429126739502
No Change Points At All
15.286771059036255
No Change Points At All
15.28966999053955
No Change Points At All
15.292142152786255
No Change Points At All
15.294806003570557
No Change Points At All
15.297259092330933
No Change Points At All
15.300031185150146
No Change Points At All
15.302511215209961
No Change Points At All
15.305169105529785
No Change Points At All
15.307646036148071
No Change Points At All
15.310277938842773
No Change Points At All
15.312727212905884
No Change Points At All
15.315377950668335
No Change Points At All
15.3179452419281
No Change Points At All
15.320595026016235
No Change Points At All
15.3230443000793

No Change Points At All
15.883884191513062
No Change Points At All
15.89615797996521
No Change Points At All
15.907276153564453
No Change Points At All
15.921044111251831
No Change Points At All
15.923784971237183
t
16.048410892486572
t
16.166692972183228
No Change Points At All
16.17931818962097
No Change Points At All
16.19115114212036
No Change Points At All
16.203186988830566
No Change Points At All
16.21462893486023
No Change Points At All
16.226099014282227
t
16.323933124542236
t
16.442044019699097
t
16.541412115097046
No Change Points At All
16.5439932346344
No Change Points At All
16.546550035476685
No Change Points At All
16.549610137939453
No Change Points At All
16.552119255065918
No Change Points At All
16.554612159729004
No Change Points At All
16.557188272476196
No Change Points At All
16.559669017791748
No Change Points At All
16.562169075012207
t
16.67509627342224
t
16.781317949295044
t
16.883085250854492
No Change Points At All
16.887109994888306
No Change Points At Al

No Change Points At All
18.080992221832275
No Change Points At All
18.08431124687195
No Change Points At All
18.086764097213745
No Change Points At All
18.08987307548523
No Change Points At All
18.092868089675903
No Change Points At All
18.09542202949524
No Change Points At All
18.097765922546387
No Change Points At All
18.100788116455078
No Change Points At All
18.10320496559143
No Change Points At All
18.105722188949585
No Change Points At All
18.10819411277771
No Change Points At All
18.110909938812256
No Change Points At All
18.11351990699768
No Change Points At All
18.115971088409424
No Change Points At All
18.11859631538391
No Change Points At All
18.12113308906555
The df is empty
18.12114405632019
No Change Points At All
18.12343716621399
The df is empty
18.123448133468628
No Change Points At All
18.125860929489136
No Change Points At All
18.12827205657959
No Change Points At All
18.13072896003723
No Change Points At All
18.133002996444702
No Change Points At All
18.135432004928

18.585927963256836
The df is empty
18.586048126220703
The df is empty
18.586059093475342
The df is empty
18.586066961288452
The df is empty
18.586073875427246
No Change Points At All
18.59024429321289
The df is empty
18.59025812149048
The df is empty
18.590266227722168
The df is empty
18.590272188186646
The df is empty
18.590277910232544
No Change Points At All
18.593297958374023
The df is empty
18.593311071395874
The df is empty
18.593317985534668
The df is empty
18.593324184417725
The df is empty
18.593330144882202
t
18.700491189956665
The df is empty
18.700560092926025
The df is empty
18.70056915283203
The df is empty
18.70057511329651
The df is empty
18.700581073760986
t
18.810660123825073
The df is empty
18.810787200927734
The df is empty
18.810832977294922
The df is empty
18.810840129852295
The df is empty
18.810846090316772
No Change Points At All
18.813834190368652
The df is empty
18.813847064971924
The df is empty
18.813853979110718
The df is empty
18.81386089324951
The df is 

In [25]:
def run_cpd_async(level, family_code):
    params = pd.read_csv(f'./Parms/{level}parms.csv', header=None, names=["Parameter", "Owner"])
    params["Parameter"] = params["Parameter"].astype('str') + f"_{levels_dict[level]}"

    # open pdf to save plots
    with PdfPages(f'./CPDResults/{level}CPDGraphs.pdf') as pdfFile:

        # initialize cpd object
        model = cpd()

        # run analysis on all parameters
        results = model.ilt_cpd_async(pp=pdfFile, 
                                    parmlist=params["Parameter"],
                                    rbf_pen=5, 
                                    TIMEDELAY=TIMEDELAY, 
                                    DAYSBACK=DAYSBACK, 
                                    fc=(family_code))
    
        for result in results:
            if result:
                # Plot result
                model.change_point_plot(pdfFile, result['df'], result['points'], result['result'], 
                                       result['parm_name'], result['cp_date'], result['cp_lot'])
#                 print(result)

        # save results to csv
        model.outputdf.merge(
            params, 
            left_on='parameter', 
            right_on='Parameter'
        ).drop(
            columns=['Parameter']
        ).rename(
            columns={1:'category'}
        ).to_csv(
            f"./CPDResults/{level.lower()}cpd_{family_code}.csv", 
            index=False
        )

In [26]:
%memit run_cpd_async("M1", "X2")

The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
No Change Points At All
The df is empty
No Change Points At All
No Change Points At AllNo Change Points At All

No Change Points At AllNo Change Points At All

The df is empty
No Change Points At All
The df is empty
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Chang

No Change Points At All
No Change Points At All
Resent Change Point Detected
No Change Points At All
Resent Change Point DetectedResent Change Point Detected

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At AllNo Change Points At All

No Change Points At AllNo Change Points At All

Resent Change Point Detected
No Change Points At All
No Change Points At All
Resent Change Point Detected
No Change Points At All
Resent Change Point Detected
Resent Change Point Detected
No Change Points At All
No Change Points At All
Resent Change Point Detected
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At AllNo Change Points At All
No Change Points At All

No Change Points At AllNo Change Points At All

No Change Points At All
No Change Points At AllN

No Change Points At All
No Change Points At All
No Change Points At AllNo Change Points At All

No Change Points At All
No Change Points At All
No Change Points At AllNo Change Points At All

No Change Points At All
Resent Change Point Detected
No Change Points At All
Resent Change Point Detected
No Change Points At All
Resent Change Point Detected
Resent Change Point Detected
No Change Points At All
No Change Points At All
Resent Change Point Detected
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
Resent Change Point DetectedNo Change Points At All

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
The df is empty
No Change Points At All
No Change Points At All
Resent Change Point Detected
Resent Change Point Detected
No Change Points At All
No Chang

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At AllNo Change Points At All

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At AllNo Change Points At All

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At AllNo Change Points At All

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
Resent Change Point Detected
No Change Points At All
Resent Change Point Detected
Resent Change Point Detected
Resent Change Point Detected
No Change Points At All
Resent Change Point Detected
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At AllNo Change Points At All

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At AllNo Change Points At All

No Change Points At All
Resent Change Point Detected
Resent Change Point Detected
No Change Points At All
Resent Change Point Detected
Resent Change Point Detected
No Change Points At

The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is emptyThe df is empty
The df is empty

The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
No Change Points At All
No Change Points At All
No Change Points At AllNo Change Points At All

The df is empty
No Change Points At All
No Change Points At All
No Change Points At AllNo Change Points At All
No Change Points At All
No Change Points At All

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At AllNo Change Points At All
No Change Points At All

No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All
No Change Points At AllNo Change Points At All
No Change Points At All
No Change Points At All
No Change Points At All

No Change Points At All
No Chang

The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is emptyThe df is empty
The df is empty
The df is empty

The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
No Change Points At All
Resent Change Point Detected
The df is empty
The df is empty
The df is empty
The df is emptyThe df is empty

The df is empty
The df is empty
Resent Change Point Detected
No Change Points At All
The df is emptyThe df is empty

The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is empty
The df is emptyThe df is empty

The df is empty
The df is empty
The df is empty
The df is emptyThe df is empty

No Change Points At All
Resent